# This notebook will use a new machine learning library not used in class to make NBA game predictions

In [1]:
# Initial imports
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier as xgbC
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Reading in game data
nba_df = pd.read_csv(Path('final_data_lr 3.csv'),parse_dates = ['GAME_DATE'], infer_datetime_format= True)
# Changing date column to an integer
nba_df['GAME_DATE'] = nba_df['GAME_DATE'].apply(lambda x: x.toordinal())
# Resetting the index
nba_df = nba_df.reset_index()
# Dropping null values
nba_df = nba_df.dropna()
# Dropping unnecessary columns
nba_df = nba_df.drop(columns= ['index'])
# Checking dtypes
nba_df.dtypes
# Displaying dataframe
display(nba_df)

,GAME_ID,SEASON_YEAR,GAME_DATE,HOME_WL,AWAY_AST_PCT,AWAY_AST_PCT_RANK,AWAY_AST_RATIO,AWAY_AST_RATIO_RANK,AWAY_AST_TO,AWAY_AST_TO_RANK,...,HOME_PIE_RANK,HOME_POSS,HOME_REB_PCT,HOME_REB_PCT_RANK,HOME_TM_TOV_PCT,HOME_TM_TOV_PCT_RANK,HOME_TS_PCT,HOME_TS_PCT_RANK,HOME_W_PCT_RANK,HOME_W_RANK
0,20100573.0,2001-02,730871,1.0,0.57355,1429.25,16.770,1219.65,1.4765,1328.10,...,1154.05,94.35,0.50320,1138.70,0.15705,1203.70,0.54820,900.35,595.50,595.50
1,20100609.0,2001-02,730875,0.0,0.57970,1380.15,16.830,1201.25,1.4685,1343.55,...,1169.35,94.40,0.50575,1091.75,0.15960,1253.35,0.54845,897.50,595.50,595.50
2,20100629.0,2001-02,730879,0.0,0.58070,1376.25,17.055,1141.75,1.4870,1314.20,...,1268.70,94.75,0.49980,1180.90,0.15900,1241.55,0.54550,933.15,654.95,654.95
3,20100661.0,2001-02,730883,1.0,0.57670,1409.30,17.145,1117.40,1.5230,1258.65,...,1334.25,94.55,0.49230,1269.15,0.15405,1182.90,0.54160,948.40,654.95,654.95
4,20100701.0,2001-02,730893,1.0,0.57295,1441.15,17.115,1127.35,1.5025,1283.25,...,1370.10,94.50,0.48900,1314.85,0.15045,1110.70,0.53645,979.95,654.95,654.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24616,22201006.0,2022-23,738590,0.0,0.57795,1251.90,17.750,1167.05,2.0220,1012.60,...,1273.70,102.50,0.48270,1313.40,0.13365,1040.40,0.56515,1353.30,719.90,719.90
24617,22201016.0,2022-23,738591,0.0,0.56940,1305.60,17.580,1202.05,2.0225,1011.25,...,1229.95,102.60,0.48635,1258.40,0.13650,1092.75,0.56880,1303.35,719.90,719.90
24618,22201028.0,2022-23,738593,0.0,0.57555,1252.00,17.830,1138.60,2.1055,946.50,...,1225.75,101.85,0.49105,1204.05,0.14560,1201.05,0.57385,1237.40,719.90,719.90
24619,22201048.0,2022-23,738596,0.0,0.57115,1288.45,17.710,1173.40,2.0960,959.50,...,1232.60,101.90,0.48325,1297.90,0.14605,1210.75,0.57000,1245.70,719.90,719.90


In [3]:
# Separate the y variable, the labels
y = nba_df[['HOME_WL']]

# Separate the X variable, the features
X = nba_df.drop(columns= ['HOME_WL', 'SEASON_YEAR'])
display(y.head())
display(X.head())

,HOME_WL
0,1.0
1,0.0
2,0.0
3,1.0
4,1.0


,GAME_ID,GAME_DATE,AWAY_AST_PCT,AWAY_AST_PCT_RANK,AWAY_AST_RATIO,AWAY_AST_RATIO_RANK,AWAY_AST_TO,AWAY_AST_TO_RANK,AWAY_DEF_RATING,AWAY_DEF_RATING_RANK,...,HOME_PIE_RANK,HOME_POSS,HOME_REB_PCT,HOME_REB_PCT_RANK,HOME_TM_TOV_PCT,HOME_TM_TOV_PCT_RANK,HOME_TS_PCT,HOME_TS_PCT_RANK,HOME_W_PCT_RANK,HOME_W_RANK
0,20100573.0,730871,0.57355,1429.25,16.770,1219.65,1.4765,1328.10,106.37,1400.60,...,1154.05,94.35,0.50320,1138.70,0.15705,1203.70,0.54820,900.35,595.50,595.50
1,20100609.0,730875,0.57970,1380.15,16.830,1201.25,1.4685,1343.55,106.43,1404.60,...,1169.35,94.40,0.50575,1091.75,0.15960,1253.35,0.54845,897.50,595.50,595.50
2,20100629.0,730879,0.58070,1376.25,17.055,1141.75,1.4870,1314.20,106.24,1392.90,...,1268.70,94.75,0.49980,1180.90,0.15900,1241.55,0.54550,933.15,654.95,654.95
3,20100661.0,730883,0.57670,1409.30,17.145,1117.40,1.5230,1258.65,105.95,1376.00,...,1334.25,94.55,0.49230,1269.15,0.15405,1182.90,0.54160,948.40,654.95,654.95
4,20100701.0,730893,0.57295,1441.15,17.115,1127.35,1.5025,1283.25,105.64,1358.95,...,1370.10,94.50,0.48900,1314.85,0.15045,1110.70,0.53645,979.95,654.95,654.95


In [4]:
# Scaling the data using MinMax scaler
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
# Define search space for RandomizedCV search
search_space = [
  {
    'clf__n_estimators': [50, 100, 150, 200],
    'clf__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'clf__max_depth': range(3, 10),
  }
] 

In [6]:
# Define cross validation
kfold = KFold(n_splits=10, random_state=42, shuffle=True)

In [7]:
# AUC and accuracy as score
scoring = {'AUC':'roc_auc', 'Accuracy':make_scorer(accuracy_score)}

In [8]:
clf_xgb = xgb.XGBClassifier()

In [9]:
# Define grid search
grid = RandomizedSearchCV(
  estimator=clf_xgb,
  param_distributions=search_space,
  cv=kfold,
  scoring=scoring,
  refit='AUC',
  verbose=1,
  n_jobs=-1
)
# Fit grid search
model = grid.fit(X_train, y_train)


Fitting 10 folds for each of 10 candidates, totalling 100 fits


/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/

[12:26:50] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.

[12:26:50] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:50] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some para

/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for ex

[12:29:05] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




...

[13:18:52] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[13:18:52] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[13:18:52] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[13:18:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Ex

/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Shane/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for ex

[13:20:50] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[13:20:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [10]:
predict = model.predict(X_test)
print('Best AUC Score: {}'.format(model.best_score_))
print('Accuracy: {}'.format(accuracy_score(y_test, predict)))
print(confusion_matrix(y_test,predict))

Best AUC Score: 0.6609311764546816
Accuracy: 0.6221572449642625
[[1184 1335]
 [ 991 2646]]


In [11]:
print(model.best_params_)

{'clf__n_estimators': 100, 'clf__max_depth': 6, 'clf__learning_rate': 0.1}


In [12]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', 
                            eval_metric="aucpr",
                            seed=42, 
                            use_label_encoder=False,
                            clf__learning_rate=0.1,
                            clf__max_depth = 6,
                            clf__n_estimators = 100)


In [13]:
clf_xgb.fit(X_train, 
            y_train,
            verbose=True, ## the next three arguments set up early stopping.
            early_stopping_rounds=10,
            eval_set=[(X_test, y_test)])

[13:52:39] WARNING: ../src/learner.cc:576: 
Parameters: { "clf__learning_rate", "clf__max_depth", "clf__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-aucpr:0.74735
[1]	validation_0-aucpr:0.75333
[2]	validation_0-aucpr:0.75333
[3]	validation_0-aucpr:0.75465
[4]	validation_0-aucpr:0.75426
[5]	validation_0-aucpr:0.75387
[6]	validation_0-aucpr:0.75425
[7]	validation_0-aucpr:0.75440
[8]	validation_0-aucpr:0.75498
[9]	validation_0-aucpr:0.75332
[10]	validation_0-aucpr:0.75287
[11]	validation_0-aucpr:0.75322
[12]	validation_0-aucpr:0.75295
[13]	validation_0-aucpr:0.75084
[14]	validation_0-aucpr:0.75044
[15]	validation_0-aucpr:0.75009
[16]	validation_0-aucpr:0.75033
[17]	validation_0-aucpr:0.75058


XGBClassifier(base_score=0.5, booster='gbtree', clf__learning_rate=0.1,
              clf__max_depth=6, clf__n_estimators=100, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='aucpr', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, ...)

In [14]:

# Making predictions
predictions = clf_xgb.predict(X_test)

In [15]:
# Checking accuracy score
accuracy_score(y_test, predictions)

0.6541585445094217

In [16]:
# Viewing classification report
print(classification_report_imbalanced(y_test, predictions)) 

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.59      0.51      0.76      0.54      0.62      0.37      2519
        1.0       0.69      0.76      0.51      0.72      0.62      0.39      3637

avg / total       0.65      0.65      0.61      0.65      0.62      0.38      6156



In [17]:
print(confusion_matrix(y_test, predictions))

[[1273 1246]
 [ 883 2754]]


In [18]:
clf_xgb.save_model('xgb_model_nba.json')